In [31]:
import pandas as pd
import numpy as np
import win32com.client
import re
import datetime
import os
from dotenv import load_dotenv
load_dotenv()
import sqlalchemy

1. ####  Create file with list of required packages

In [2]:
with open('requirements.txt', 'w') as f:
    f.write('\n'.join(f'{m.__name__}=={m.__version__}' for m in globals().values() if getattr(m, '__version__', None)))

2. #### Downlaod email attachment

In [3]:
def read_email():
    outlook_mail_client = win32com.client.Dispatch('Outlook.Application').GetNamespace('MAPI')
    inbox = outlook_mail_client.GetDefaultFolder(6)
    messages = inbox.Items
    path = 'E:/ACTIVE/Python/experian_data-service'
    message = messages.GetFirst()
    today_date = str(datetime.date.today())
    while True:
        try:
            email_subject = 'SUBJECT-TITLE'
            subject = str(message.Subject).lower()

           # Search email with subject

            if re.search(email_subject, subject) != None:
                attachments = message.Attachments
                attachment = str(attachments.Item(1)).lower()
                attachment.SaveASFile(path + '\\' + attachment_name)
            else:
                pass
            message = messages.GetNext()
        except:
            message = messages.GetNext()
    exit

In [4]:
#read_email()

4. #### Read IBRD dataset

In [5]:
def read_data(filename):
    if filename.endswith('csv'):
        df = pd.read_csv(filename)
    return df

In [7]:
filepath = os.getenv('file_path')
filename = os.environ.get('file_name')
data =  read_data(filepath+filename)

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8986 entries, 0 to 8985
Data columns (total 33 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   End of Period                 8986 non-null   object 
 1   Loan Number                   8986 non-null   object 
 2   Region                        8986 non-null   object 
 3   Country Code                  8984 non-null   object 
 4   Country                       8986 non-null   object 
 5   Borrower                      8929 non-null   object 
 6   Guarantor Country Code        8704 non-null   object 
 7   Guarantor                     8706 non-null   object 
 8   Loan Type                     8986 non-null   object 
 9   Loan Status                   8986 non-null   object 
 10  Interest Rate                 8865 non-null   float64
 11  Currency of Commitment        0 non-null      float64
 12  Project ID                    8660 non-null   object 
 13  Pro

In [19]:
# 303 loans with zero principal
zero_principal = data[data["Original Principal Amount"] == 0].shape[0]
zero_principal


303

In [20]:
# 2437 loans with zero interest
zero_interest = data[data["Interest Rate"] == 0].shape[0]
zero_interest

2437

In [23]:
#Loan type and status summary
loan_status = data['Loan Status'].value_counts()
loan_type= data['Loan Type'].value_counts()

5. #### Data cleaning

In [24]:
# Convert date time fields from  object data type to datetime data type
date_fields = ['End of Period', 'First Repayment Date', 'Last Repayment Date', 'Agreement Signing Date', 'Board Approval Date', 'Effective Date (Most Recent)', 'Closed Date (Most Recent)', 'Last Disbursement Date']
for field in date_fields:
    data[field] = pd.to_datetime(data[field])

In [28]:
data.head(2)

,End of Period,Loan Number,Region,Country Code,Country,Borrower,Guarantor Country Code,Guarantor,Loan Type,Loan Status,...,Repaid 3rd Party,Due 3rd Party,Loans Held,First Repayment Date,Last Repayment Date,Agreement Signing Date,Board Approval Date,Effective Date (Most Recent),Closed Date (Most Recent),Last Disbursement Date
0,2020-11-30,IBRD00010,EUROPE AND CENTRAL ASIA,FR,France,CREDIT NATIONAL,FR,France,NPL,Fully Repaid,...,249962000.0,0.0,0.0,1952-11-01,1977-05-01,1947-05-09,1947-05-09,1947-06-09,1947-12-31,NaT
1,2020-11-30,IBRD00020,EUROPE AND CENTRAL ASIA,NL,Netherlands,NaN,NaN,NaN,NPL,Fully Repaid,...,87672000.0,0.0,0.0,1952-04-01,1972-10-01,1947-08-07,1947-08-07,1947-09-11,1948-03-31,NaT


In [29]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8986 entries, 0 to 8985
Data columns (total 33 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   End of Period                 8986 non-null   datetime64[ns]
 1   Loan Number                   8986 non-null   object        
 2   Region                        8986 non-null   object        
 3   Country Code                  8984 non-null   object        
 4   Country                       8986 non-null   object        
 5   Borrower                      8929 non-null   object        
 6   Guarantor Country Code        8704 non-null   object        
 7   Guarantor                     8706 non-null   object        
 8   Loan Type                     8986 non-null   object        
 9   Loan Status                   8986 non-null   object        
 10  Interest Rate                 8865 non-null   float64       
 11  Currency of Commitment        

In [37]:
engine_stmt = 'mysql+pymysql://root:@localhost/test'
engine = sqlalchemy.create_engine(engine_stmt)
m = data.to_sql('experian', engine, if_exists='replace', index=False)

In [38]:
m